# LangGraph
`pip install -U langgraph`
## Get started

In [1]:
# pip install -qU "langchain[anthropic]" to call the model
from langgraph.prebuilt import create_react_agent
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
            model="Qwen/Qwen3-8B",
            # model="Qwen/Qwen3-30B-A3B-Instruct-2507",
            api_key="sk-jvjyawqpodlkxlywatvemcdykkrbvthhjyjyapyvtnifwlbl",
            base_url="https://api.siliconflow.cn/v1/",
        )

def get_weather(city: str) -> str:
    """Get weather for a given city."""
    return f"It's always sunny in {city}!"

agent = create_react_agent(
    model=model,
    tools=[get_weather],
    prompt="You are a helpful assistant"
)

# Run the agent
agent.invoke({"messages": [{"role": "user", "content": "what is the weather in sf"}]})

{'messages': [HumanMessage(content='what is the weather in sf', additional_kwargs={}, response_metadata={}, id='0e751d70-5c03-4d37-b418-d405e28f1409'),
  AIMessage(content='\n\n', additional_kwargs={'tool_calls': [{'id': '0198e0adf6f3ded6d2d1fa3624d36b1a', 'function': {'arguments': ' {"city": "San Francisco"}', 'name': 'get_weather'}, 'type': 'function', 'index': 0}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 126, 'prompt_tokens': 162, 'total_tokens': 288, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': None, 'reasoning_tokens': 117, 'rejected_prediction_tokens': None}, 'prompt_tokens_details': None}, 'model_name': 'Qwen/Qwen3-8B', 'system_fingerprint': '', 'id': '0198e0ade71eae4bf23e9b78fab9e15b', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--d8fd72e4-2e2d-45d5-81c5-e0e7d324958e-0', tool_calls=[{'name': 'get_weather', 'args': {'city': 'San Francisco'}, 'id': '0198e0adf6f3ded6d2d1fa3624

## LangGraph quickstart
...  
### Add a custom prompt
- 静态：字符串被解释为系统消息。
- 动态：在运行时基于输入或配置生成的消息列表。

In [2]:
from langgraph.prebuilt import create_react_agent
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
            model="Qwen/Qwen3-8B",
            # model="Qwen/Qwen3-30B-A3B-Instruct-2507",
            api_key="sk-jvjyawqpodlkxlywatvemcdykkrbvthhjyjyapyvtnifwlbl",
            base_url="https://api.siliconflow.cn/v1/",
        )

def get_weather(city: str) -> str:
    """Get weather for a given city."""
    return f"It's always sunny in {city}!"

In [ ]:
# 静态提示词
agent = create_react_agent(
    model=model,
    tools=[get_weather],
    # prompt="You are a helpful assistant", # default
    prompt="Never answer questions about the weather."
)


# Run the agent
agent.invoke({"messages": [{"role": "user", "content": "what is the weather in sf"}]})

{'messages': [HumanMessage(content='what is the weather in sf', additional_kwargs={}, response_metadata={}, id='31ee20ea-6424-4a86-aaf1-665f36eeba80'),
  AIMessage(content='\n\n', additional_kwargs={'tool_calls': [{'id': '0198e0ae2812cb9f2ec80ae0ee0560fc', 'function': {'arguments': ' {"city": "SF"}', 'name': 'get_weather'}, 'type': 'function', 'index': 0}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 76, 'prompt_tokens': 163, 'total_tokens': 239, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': None, 'reasoning_tokens': 68, 'rejected_prediction_tokens': None}, 'prompt_tokens_details': None}, 'model_name': 'Qwen/Qwen3-8B', 'system_fingerprint': '', 'id': '0198e0ae1937873f7c8747c70cc83d34', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--c360f72d-74f9-481a-8817-b7a319ea5491-0', tool_calls=[{'name': 'get_weather', 'args': {'city': 'SF'}, 'id': '0198e0ae2812cb9f2ec80ae0ee0560fc', 'type': 'tool_c

In [4]:
# 动态提示词
from langchain_core.messages import AnyMessage
from langchain_core.runnables import RunnableConfig
from langgraph.prebuilt.chat_agent_executor import AgentState
from langgraph.prebuilt import create_react_agent

def get_weather(city: str) -> str:
    """Get weather for a given city."""
    return f"It's always sunny in {city}!"

def prompt(state: AgentState, config: RunnableConfig) -> list[AnyMessage]:  
    user_name = config["configurable"].get("user_name")
    system_msg = f"You are a helpful assistant. Address the user as {user_name}."
    return [{"role": "system", "content": system_msg}] + state["messages"]

from langchain_openai import ChatOpenAI

model = ChatOpenAI(
            model="Qwen/Qwen3-8B",
            # model="Qwen/Qwen3-30B-A3B-Instruct-2507",
            api_key="sk-jvjyawqpodlkxlywatvemcdykkrbvthhjyjyapyvtnifwlbl",
            base_url="https://api.siliconflow.cn/v1/",
        )

agent = create_react_agent(
    model=model,
    tools=[get_weather],
    prompt=prompt
)

agent.invoke(
    {"messages": [{"role": "user", "content": "what is the weather in sf"}]}, # input=state
    config={"configurable": {"user_name": "John Smith"}}
)

{'messages': [HumanMessage(content='what is the weather in sf', additional_kwargs={}, response_metadata={}, id='bff3b94c-1bf2-448a-90d0-1b0b19b48223'),
  AIMessage(content='\n\n', additional_kwargs={'tool_calls': [{'id': '0198e0ae568056029c456a28ac836faf', 'function': {'arguments': ' {"city": "San Francisco"}', 'name': 'get_weather'}, 'type': 'function', 'index': 0}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 110, 'prompt_tokens': 169, 'total_tokens': 279, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': None, 'reasoning_tokens': 101, 'rejected_prediction_tokens': None}, 'prompt_tokens_details': None}, 'model_name': 'Qwen/Qwen3-8B', 'system_fingerprint': '', 'id': '0198e0ae44598890bb47cfd01dd175f6', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--352d3424-b10f-4c09-90b5-1b909227ddf4-0', tool_calls=[{'name': 'get_weather', 'args': {'city': 'San Francisco'}, 'id': '0198e0ae568056029c456a28ac

### Add memory
要允许与代理进行多轮对话，您需要在创建代理时提供检查指针来启用持久性。在运行时，您需要提供一个包含 thread_id 的配置-对话（会话）的唯一标识符：

In [5]:
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import InMemorySaver

# 当您启用检查指针时，它将在提供的检查指针数据库（或内存中，如果使用 InMemorySaver ）中的每一步存储代理状态。
checkpointer = InMemorySaver()

agent = create_react_agent(
    model=model,
    tools=[get_weather],
    checkpointer=checkpointer  
)

# Run the agent
config = {"configurable": {"thread_id": "1"}}
sf_response = agent.invoke(
    {"messages": [{"role": "user", "content": "what is the weather in sf"}]},
    config  
)
ny_response = agent.invoke(
    {"messages": [{"role": "user", "content": "what about new york?"}]},
    # 当使用相同的 thread_id 第二次调用代理时，将自动包括第一次对话的原始消息历史记录以及新的用户输入。
    config
)
sf_response, ny_response

({'messages': [HumanMessage(content='what is the weather in sf', additional_kwargs={}, response_metadata={}, id='630acff5-111a-40e4-b2cf-3c7aa9e33c45'),
   AIMessage(content='\n\n', additional_kwargs={'tool_calls': [{'id': '0198e0aec1708ddddea4615d79146a44', 'function': {'arguments': ' {"city": "San Francisco"}', 'name': 'get_weather'}, 'type': 'function', 'index': 0}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 181, 'prompt_tokens': 156, 'total_tokens': 337, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': None, 'reasoning_tokens': 172, 'rejected_prediction_tokens': None}, 'prompt_tokens_details': None}, 'model_name': 'Qwen/Qwen3-8B', 'system_fingerprint': '', 'id': '0198e0aea8eb0b14fa19c61b2e6fc253', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--706537bc-61e8-4151-a774-62d16cbfd29c-0', tool_calls=[{'name': 'get_weather', 'args': {'city': 'San Francisco'}, 'id': '0198e0aec1708ddddea4615d

### Configure structured output

In [3]:
from pydantic import BaseModel
from langgraph.prebuilt import create_react_agent
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
        model="Qwen/Qwen2.5-72B-Instruct",
        # 硅基流动
        # api_key="sk-jvjyawqpodlkxlywatvemcdykkrbvthhjyjyapyvtnifwlbl",
        # base_url="https://api.siliconflow.cn/v1/",
        # modelscope
        base_url="https://api-inference.modelscope.cn/v1/",
        api_key="ms-e2666046-2f3b-4c76-bcc0-e21f8ebf9ea1",
    )

def get_weather(city: str) -> str:
    """Get weather for a given city."""
    return f"It's always sunny in {city}!"

class WeatherResponse(BaseModel):
    city: str
    conditions: str
    
agent = create_react_agent(
    model=model,
    tools=[get_weather],
    response_format=WeatherResponse  
)
json_prompt = """\nPlease respond in JSON format.
{
    "city": "city of question",
    "conditions": "conditions of city"
}
"""
response = agent.invoke(
    {"messages": [{"role": "user", "content": "what is the weather in sf?" + json_prompt}]}
)

response["structured_response"]

ValidationError: 1 validation error for WeatherResponse
  Invalid JSON: expected ident at line 3 column 2 [type=json_invalid, input_value=' \n\nNote: The actual we...eliable weather source.', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/json_invalid

In [2]:
from pydantic import BaseModel
from langchain_core.messages import AnyMessage
from langchain_core.runnables import RunnableConfig
from langgraph.prebuilt.chat_agent_executor import AgentState
from langgraph.prebuilt import create_react_agent
from langchain_openai import ChatOpenAI

def get_weather(city: str) -> str:
    """Get weather for a given city."""
    return f"It's always sunny in {city}!"

def prompt(state: AgentState, config: RunnableConfig) -> list[AnyMessage]:  
    user_name = config["configurable"].get("user_name")
    system_msg = f"You are a helpful assistant. Address the user as {user_name}."
    return [{"role": "system", "content": system_msg}] + state["messages"]

model = ChatOpenAI(
        model="Qwen/Qwen2.5-72B-Instruct",
        # 硅基流动
        api_key="sk-jvjyawqpodlkxlywatvemcdykkrbvthhjyjyapyvtnifwlbl",
        base_url="https://api.siliconflow.cn/v1/",
        # # modelscope 
        # base_url="https://api-inference.modelscope.cn/v1/",
        # api_key="ms-e2666046-2f3b-4c76-bcc0-e21f8ebf9ea1",
    )

class WeatherResponse(BaseModel):
    city: str
    conditions: str
    
agent = create_react_agent(
    model=model,
    tools=[get_weather],
    # response_format=WeatherResponse
    response_format=(prompt, WeatherResponse),
)

response = agent.invoke(
    # {"messages": [{"role": "user", "content": "what is the weather in sf?\nPlease respond in JSON format.\n{\"city\": \"xxx\", \"conditions\": \"xxx\"}"}]}
    {"messages": [{"role": "user", "content": "what is the weather in sf?"}]}
)

response["structured_response"]
# BadRequestError: Error code: 400 - {'code': 20033, 'message': 'Prefix can not work with json mode.', 'data': None}

ValidationError: 2 validation errors for SystemMessage
content.str
  Input should be a valid string [type=string_type, input_value=<function prompt at 0x10b2019e0>, input_type=function]
    For further information visit https://errors.pydantic.dev/2.11/v/string_type
content.list[union[str,dict[any,any]]]
  Input should be a valid list [type=list_type, input_value=<function prompt at 0x10b2019e0>, input_type=function]
    For further information visit https://errors.pydantic.dev/2.11/v/list_type